In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

import sys
sys.path.append("../../")
import pandas as pd
import os
import numpy as np
import DeepStrain.functions_collection as ff
from tqdm.auto import tqdm


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, hidden_size = 128):
        super(LogisticRegression, self).__init__()
        self.input_size = input_size
        self.hidden = nn.Linear(input_size, hidden_size)  # Hidden layer
        self.output = nn.Linear(hidden_size, 1)  # Output layer for binary classification

    def forward(self, x):
        x = torch.sigmoid(self.hidden(x))  # Sigmoid activation for hidden layer
        out = torch.sigmoid(self.output(x))  # Sigmoid activation for output layer
        return out

class Trainer():
    def __init__(
        self,
        regression_model,
        train_num_steps,
        save_folder,

        train_lr = 1e-3,
        train_lr_decay_every = 1,
        save_models_every = 2,):
        

        super().__init__()
        self.model = regression_model
        self.input_size = regression_model.input_size
        self.save_folder = save_folder; os.makedirs(self.save_folder, exist_ok=True)

        # loss:
        self.criterion = nn.BCELoss()

        # optimizer
        self.opt = Adam(self.model.parameters(), lr = train_lr, betas = (0.9, 0.99))
        # self.opt = optim.SGD(model.parameters(), lr=0.001)
        self.scheduler = StepLR(self.opt, step_size = 1, gamma=0.95)
        self.train_lr_decay_every = train_lr_decay_every
        self.step = 0
        self.train_num_steps = train_num_steps
        self.save_models_every = save_models_every

    def simple_val(self,trained_model, x, y):
        y_pred = []; y_pred_float = []

        for index in range(0,x.shape[0]):
            new_data = x[index,:]

            trained_model.eval()

            with torch.no_grad():
                prediction = trained_model(new_data); y_pred_float.append(prediction.item())
                predicted_class = 1 if prediction.item() > 0.5 else 0; y_pred.append(predicted_class)

        y_pred = np.asarray(y_pred); y_pred_float = np.asarray(y_pred_float); y_true = torch.clone(y).numpy()

        # calculate accuracy using predict_collect and ground truth
        accuracy, sensitivity, specificity,_,_,_,_ = ff.quantitative(y_pred, y_true)
        return y_pred, y_pred_float, accuracy, sensitivity, specificity

    def save_model(self, stepNum):
        data = {
            'step': self.step,
            'model': self.model.state_dict(),
            'opt': self.opt.state_dict(),
            'scheduler': self.scheduler.state_dict(),}
        os.makedirs(os.path.join(self.save_folder, 'models'), exist_ok=True)
        torch.save(data, os.path.join(self.save_folder, 'models', 'model-' + str(stepNum) + '.pt'))

    def load_model(self, trained_model_filename):
        data = torch.load(trained_model_filename)

        self.model.load_state_dict(data['model'])

        self.step = data['step']
        self.opt.load_state_dict(data['opt'])
        self.scheduler.load_state_dict(data['scheduler'])


    def train(self, X_train, Y_train, X_val, Y_val, pre_trained_model = None ,start_step = None):
    
        training_log = []

        # load pre-trained
        if pre_trained_model is not None:
            self.load_model(pre_trained_model)
            print('model loaded from ', pre_trained_model)

        if start_step is not None:
            self.step = start_step

        with tqdm(initial = self.step, total = self.train_num_steps) as pbar:
            
            while self.step < self.train_num_steps:
                print('training epoch: ', self.step + 1)
                print('learning rate: ', self.scheduler.get_last_lr()[0])

                self.opt.zero_grad()

                # Forward pass
                outputs = self.model(X_train)

                # Calculate loss
                loss = self.criterion(outputs.view(-1), Y_train.float())  # BCELoss expects float inputs

                loss.backward()
                self.opt.step()

                self.step += 1

                # save the model
                if self.step !=0 and (self.step % self.save_models_every == 0):
                   self.save_model(self.step)
                
                if self.step !=0 and (self.step % self.train_lr_decay_every == 0):
                    self.scheduler.step()

                if self.step !=0 and (self.step % self.save_models_every == 0):
                    _,_, train_accuracy, train_sensitivity, train_specificity = self.simple_val(self.model, X_train, Y_train)
                    _,_, val_accuracy, val_sensitivity, val_specificity = self.simple_val(self.model, X_val, Y_val)
                    print('epoch is: ', self.step, ' train loss: ', loss.item(), ' train accuracy: ', train_accuracy, ' sensitivity: ', train_sensitivity, ' specificity: ', train_specificity, ' val accuracy: ', val_accuracy, ' sensitivity: ', val_sensitivity, ' specificity: ', val_specificity) 
                            
                    # save the training log
                    training_log.append([self.step,self.scheduler.get_last_lr()[0], loss.item(), train_accuracy, train_sensitivity, train_specificity, val_accuracy, val_sensitivity, val_specificity])
                    df = pd.DataFrame(training_log,columns = ['iteration','learning_rate', 'train_loss', 'train_accuracy', 'train_sensitivity', 'train_specificity', 'val_accuracy', 'val_sensitivity', 'val_specificity'])
                    log_folder = os.path.join(self.save_folder,'log');ff.make_folder([log_folder])
                    df.to_excel(os.path.join(log_folder, 'training_log.xlsx'),index=False)

                pbar.update(1)




In [49]:
class simple_DataLoader():
    def __init__(self, spreadsheet_path, data_path, index_range):
        
        super().__init__()
        self.spreadsheet = pd.read_excel(spreadsheet_path)
        self.data_path = data_path
        self.index_range = index_range

        self.patient_list = self.spreadsheet.iloc[self.index_range]

    def load_X(self):
        X = []
        for patient_index in range(0,self.patient_list.shape[0]):
            patient_id_num = self.patient_list['OurID'].iloc[patient_index]
            patient_id = ff.XX_to_ID_00XX(patient_id_num)

            # load strain first:
            strain_folder = os.path.join(self.data_path, 'results/strain', patient_id)
            # global strain
            global_strain = np.load(os.path.join(strain_folder, 'global_strain.npy'))
            global_radial_strain = np.asarray(global_strain[0]).reshape(1,-1)
            global_circumferential_strain = np.asarray(global_strain[1]).reshape(1,-1)

            # layer strain
            layer_strain = np.load(os.path.join(strain_folder, 'layer_strain.npy'))
            layer_radial_strain = layer_strain[0,:].reshape(1,-1)
            layer_circumferential_strain = layer_strain[1,:].reshape(1,-1)

            # polar strain
            polar_strain = np.load(os.path.join(strain_folder, 'polar_strain.npy'), allow_pickle=True)
            Ecc_aha = np.asarray(polar_strain[5][:-1]).reshape(1,-1)
            Err_aha = np.asarray(polar_strain[7][:-1]).reshape(1,-1)

            # WTCI
            wtci = np.load(os.path.join(strain_folder, 'wtci.npy'), allow_pickle=True)
            wtci_aha = np.asarray(wtci[4][:-1]).reshape(1,-1)

            # load geometry then:
            geometry_folder = os.path.join(self.data_path, 'results/geometry', patient_id)
            # circular index
            circular_index = np.load(os.path.join(geometry_folder, 'circular_index_collect.npy')).reshape(1,-1)
            # centers
            centers = np.load(os.path.join(geometry_folder, 'centers_collect.npy'), allow_pickle=True)
            center_delta_to_base = centers[1][1:,:]
            # euclidean distance from center_delta_to_base, first row is x and second row is y
            center_delta_to_base_euclidean = np.sqrt(np.sum(np.square(center_delta_to_base), axis=1)).reshape(1,-1)
            # enclosed_area = np.asarray(centers[-1]).reshape(1,-1)

            # axis len
            axis = np.load(os.path.join(geometry_folder, 'axis_len_collect.npy'), allow_pickle=True)
            major_axis = axis[:-1,0].reshape(1,-1)
            minor_axis = axis[:-1,1].reshape(1,-1)

            # concatenate all features
            features = np.concatenate((global_radial_strain,   # 0
                                       global_circumferential_strain,  # 1
                                    layer_radial_strain,  #2:5
                                    layer_circumferential_strain, #5:8
                                    Ecc_aha, #8:24
                                    np.concatenate((Err_aha[:,0:12] , wtci_aha[:,12:16]),axis = 1), #24:40

                                    # geometry
                                    circular_index, #40:50
                                    center_delta_to_base_euclidean, #50:59
                                    major_axis, #59:69
                                    minor_axis), #69:79
                                    axis=1)
            X.append(features)
        X = np.squeeze(np.asarray(X))
        return torch.from_numpy(X).float()

    def load_Y(self):
        Y = self.spreadsheet['label_for_ML'].iloc[self.index_range]
        return torch.from_numpy(Y.to_numpy()).float()
        

# Main script: 

In [50]:
# load data
spreadsheet_path = '/mnt/mount_zc_NAS/HFpEF/data/HFpEF_data/Patient_list/Important_HFpEF_Patient_list_unique_patient_w_readmission_finalized.xlsx' 
data_path = '/mnt/mount_zc_NAS/Deepstrain'
index_range = range(0,50)
dataloader = simple_DataLoader( spreadsheet_path, data_path, index_range)
X = dataloader.load_X()
Y = dataloader.load_Y()

In [51]:
print(X.shape)

torch.Size([50, 79])


In [53]:
# main script: 
save_folder = '/mnt/mount_zc_NAS/Deepstrain/HFpEF_analysis/models/logistic_regression/'
X_train, Y_train,  train_idx, X_val, Y_val, val_idx = ff.split_train_val(X, Y, 5, 0 , save_split_file = '/mnt/mount_zc_NAS/Deepstrain/HFpEF_analysis/models/logistic_regression/data_split.npy')
print('train idx: ', train_idx, ' val idx: ', val_idx)
model = LogisticRegression(input_size = X_train.shape[-1])
trainer = Trainer(model,  train_num_steps = 200,
                  save_folder = save_folder,
                  train_lr = 1e-3,  train_lr_decay_every = 100,   save_models_every = 100,)

trainer.train(X_train, Y_train, X_val, Y_val, pre_trained_model = None ,start_step = None)


train idx:  [ 5  6  7 11 12  8  9 10 17 20 13 14 15 22 25 16 18 19 27 33 21 23 24 34
 37 26 28 29 39 40 30 31 32 35 41 36 38 42 43 46 44 45 47 49 48]  val idx:  [1 2 3 0 4]


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 1/200 [00:00<00:25,  7.94it/s]

training epoch:  1
learning rate:  0.001
training epoch:  2
learning rate:  0.001


  2%|▏         | 3/200 [00:00<00:19, 10.29it/s]

...

  6%|▌         | 12/200 [00:01<00:20,  9.23it/s]

training epoch:  12
learning rate:  0.001
training epoch:  13
learning rate:  0.001


  7%|▋         | 14/200 [00:01<00:20,  8.96it/s]


training epoch:  14
learning rate:  0.001
training epoch:  15
learning rate:  0.001


KeyboardInterrupt: 